In [1]:
from ddganAE.models import AAE, AAE_combined_loss, CAE, SVDAE
from ddganAE.architectures.D3 import *
from ddganAE.preprocessing import convert_2d
from keras.regularizers import l2
from ddganAE.utils import calc_pod
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

# POD

### Data preprocessing

In [2]:
grids = np.load("./../submodules/DD-GAN/data/processed/sf_dataset.npy")
# Set all >0 to 0 and all <1 to 1 for alpha field
grids[:,:,:,:,3][np.where(grids[:,:,:,:,3] < 0)] = 0
grids[:,:,:,:,3][np.where(grids[:,:,:,:,3] > 1)] = 1

In [3]:
# Rescale all the velocity fields
scaler = MinMaxScaler()
grids[:,:,:,:,:3] = scaler.fit_transform(grids[:,:,:,:,:3].reshape(-1, 1)).reshape(grids[:,:,:,:,:3].shape)

In [4]:
grids_long = np.moveaxis(np.moveaxis(grids, 4, 0), 1, 4).reshape((grids.shape[-1], grids.shape[1]*grids.shape[2]*grids.shape[3], grids.shape[0]))

### Do actual pod

In [5]:
coeffs, R, s = calc_pod(grids_long, nPOD=10)

### Check reconstruction loss

In [10]:
# Reconstruct the first subgrid
recon = R @ coeffs[0]
print(recon, '\n',
      grids_long[0, :, :])

np.allclose(recon, grids_long[0, :, :], atol=0.4)

[[0.3552     0.36203752 0.36380589 ... 0.30232951 0.30877547 0.30857463]
 [0.35514146 0.36200387 0.36393034 ... 0.30231973 0.30882809 0.30857821]
 [0.3550856  0.36196616 0.36405083 ... 0.30231788 0.30888758 0.30858607]
 ...
 [0.87383535 0.86640348 0.86203201 ... 0.77366286 0.78064649 0.77859742]
 [0.87373268 0.86612371 0.8619513  ... 0.77379807 0.7809243  0.77871433]
 [0.87369322 0.86589358 0.86187987 ... 0.77392585 0.78117842 0.77883699]] 
 [[0.3593186  0.35875437 0.35910755 ... 0.3149745  0.31578264 0.28099415]
 [0.35933733 0.3588193  0.35914356 ... 0.31509516 0.3158377  0.28099415]
 [0.35935605 0.35888425 0.3591796  ... 0.31521606 0.31589282 0.28099415]
 ...
 [0.87558043 0.8758235  0.87582356 ... 0.6969149  0.6205108  0.78142804]
 [0.8755887  0.8758235  0.87582344 ... 0.6968136  0.6205108  0.7809142 ]
 [0.875596   0.8758235  0.8758235  ... 0.6967258  0.6205108  0.7804715 ]]


True

In [23]:
# Calculate MSE
mean = 0
recon_grid = np.zeros(grids_long.shape)
for j in range(4):
    recon = R @ coeffs[j]
    for i in range(100):
        mean += tf.keras.losses.MSE(recon[:, i], grids_long[j, :, i]).numpy()/100
    recon_grid[j, :, :] = recon
print(mean)

0.0019314816678883053


### Reconstruct back to original shape

In [26]:
grids_recon = np.swapaxes(recon_grid, 0, 2).reshape(*grids.shape)

In [30]:
np.allclose(grids, grids_recon, atol=0.4)

True

# Convolutional Autoencoder

### Data preprocessing

In [2]:
grids = np.load("./../submodules/DD-GAN/data/processed/sf_dataset.npy")

In [3]:
grids.shape

(100, 20, 20, 60, 4)

In [5]:
# Set all >0 to 0 and all <1 to 1 for alpha field
grids[:,:,:,:,3][np.where(grids[:,:,:,:,3] < 0)] = 0
grids[:,:,:,:,3][np.where(grids[:,:,:,:,3] > 1)] = 1

In [6]:
# Rescale all the velocity fields
scaler = MinMaxScaler()
grids[:,:,:,:,:3] = scaler.fit_transform(grids[:,:,:,:,:3].reshape(-1, 1)).reshape(grids[:,:,:,:,:3].shape)

In [7]:
x_train, x_val = train_test_split(grids, test_size=0.1)

### Model setup

In [8]:
initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
optimizer = tf.keras.optimizers.Nadam(lr=0.00005, beta_1=0.9, beta_2=0.999)

input_shape = (20, 20, 60, 4)

encoder, decoder = build_omata_encoder_decoder(input_shape, 10, initializer, info=True, act='elu', dense_act='relu')

cae = CAE(encoder, decoder, optimizer)
cae.compile(input_shape)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_7 (Conv3D)            (None, 20, 20, 60, 16)    1744      
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 10, 10, 30, 16)    0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 10, 10, 30, 8)     3464      
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 5, 5, 15, 8)       0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 5, 5, 15, 8)       1736      
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 3, 3, 8, 8)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)              

In [9]:
cae.train(x_train, 100, val_data=x_val, batch_size=2)

# Adversarial Autoencoder